In [9]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForCausalLM
import torch

In [13]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer
model_name = "meta-llama/Llama-3.1-8B"
model = TFAutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

c:\Users\ngmai\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
models = {
    "Llama-3.1-8B": {
        "model": TFAutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B"),
        "tokenizer": AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B"),
    },
    "Qwen-2.5-7B": {
        "model": TFAutoModelForCausalLM.from_pretrained("Qwen/Qwen2-VL-7B-Instruct"),
        "tokenizer": AutoTokenizer.from_pretrained("Qwen/Qwen2-VL-7B-Instruct"),
    },
    "URA-LLama-7B": {
        "model": TFAutoModelForCausalLM.from_pretrained("ura-hcmut/ura-llama-7b-r64"),
        "tokenizer": AutoTokenizer.from_pretrained("ura-hcmut/ura-llama-7b-r64"),
    },
}

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B.
401 Client Error. (Request ID: Root=1-672bc985-459e4f357c10b5b821dc3bc3;a022bbeb-8a67-4f40-adb5-d34fd5a0b2cf)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model_name, model_data in models.items():
    model_data["model"].to(device)

In [ ]:
def prompt_model(model, tokenizer, prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
def tree_of_thoughts(model_data, claim, evidence, depth=3):
    """
    Tree of Thoughts: Recursive reasoning with depth to confirm stance.
    """
    model, tokenizer = model_data["model"], model_data["tokenizer"]
    initial_prompt = f"Claim: {claim}\nEvidence: {evidence}\nWhat is the stance on this claim?"
    response = prompt_model(model, tokenizer, initial_prompt)

    thoughts = [response]
    for _ in range(depth - 1):
        new_prompt = f"{initial_prompt}\nPrevious reasoning: {thoughts[-1]}\nProvide more reasoning based on the previous response."
        response = prompt_model(model, tokenizer, new_prompt)
        thoughts.append(response)

    return thoughts[-1]

In [ ]:
def generate_counterfactual(claim):
    # Simple counterfactual generation by negating the claim (this can be improved with NLP techniques)
    counterfactual = claim.replace("is", "is not") if "is" in claim else f"Not: {claim}"
    return counterfactual

In [ ]:
def counterfactual_prompting(model_data, claim, evidence):
    """
    Prompt model with both claim and counterfactual to check consistency.
    """
    model, tokenizer = model_data["model"], model_data["tokenizer"]
    counterfactual_claim = generate_counterfactual(claim)

    # Original claim
    original_response = tree_of_thoughts(model_data, claim, evidence)

    # Counterfactual claim
    counterfactual_response = tree_of_thoughts(model_data, counterfactual_claim, evidence)

    # Check consistency
    if original_response != counterfactual_response:
        print("Inconsistent responses. Model requires further prompting.")
        return "Inconsistent"
    return original_response

In [ ]:
def ensemble_prediction(claim, evidence, models, method="majority_vote"):
    responses = {}
    for model_name, model_data in models.items():
        print(f"Processing with {model_name}")
        response = counterfactual_prompting(model_data, claim, evidence)
        responses[model_name] = response

    # Majority voting for stance prediction
    if method == "majority_vote":
        response_counts = {}
        for response in responses.values():
            response_counts[response] = response_counts.get(response, 0) + 1
        final_response = max(response_counts, key=response_counts.get)
    else:
        final_response = "Custom ensemble not implemented"
    
    return final_response


In [ ]:
# Load dataset (assuming CSV format)
train_data = pd.read_csv("/mnt/data/train_final.csv")
dev_data = pd.read_csv("/mnt/data/dev_final.csv")
test_data = pd.read_csv("/mnt/data/test_final.csv")

In [ ]:
# Example usage on a sample from the dataset
for _, row in test_data.iterrows():
    claim = row["claim_column"]  # Replace with actual column name
    evidence = row["evidence_column"]  # Replace with actual column name
    label = row["label_column"]  # Replace with actual column name
    
    predicted_stance = ensemble_prediction(claim, evidence, models)
    print(f"Claim: {claim}")
    print(f"Predicted Stance: {predicted_stance}")
    print(f"Actual Label: {label}")
    print("-" * 50)